# Segmenting cell images

We start with the same imports as last time

In [ ]:
%matplotlib inline
import mahotas as mh
import numpy as np
from matplotlib import pyplot as plt
from IPython.html.widgets import interact, fixed
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 10 x 8 inches
plt.gray()

mahotas ships with an image of nuclei for our analysis.

Because of file format issues (many image loaders cannot correctly handle single channel PNG images), it is encoded as an RGB image, where all the channels have the same value. So, we convert it to a single channel image (using `max`, but `min` would give the same result):

In [ ]:
dna = mh.demos.load('nuclear')
print(dna.shape)
dna = dna.max(axis=2)
print(dna.shape)
plt.imshow(dna)

## Thresholding

The first thing we try is Otsu thresholding, a venerable method in image processing:

In [ ]:
T_otsu = mh.otsu(dna)
print(T_otsu)
plt.imshow(dna > T_otsu)

Does not work so well on this data. What does work well is thresholding by the mean (this actually works well not just on this image, but for other similar images too, see [this paper of mine](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5193098&url=http%3A%2F%2Fieeexplore.ieee.org%2Fxpls%2Fabs_all.jsp%3Farnumber%3D5193098):

In [ ]:
T_mean = dna.mean()
print(T_mean)
plt.imshow(dna > T_mean)

Only issue is that the results are very noisy around the edges, but we can add some Gaussian blurring to correct for this:

In [ ]:
dnaf = mh.gaussian_filter(dna, 2.)
T_mean = dnaf.mean()
bin_image = dnaf > T_mean
plt.imshow(bin_image)

Right now, we have a binary image, but it is useful to [label the image](http://en.wikipedia.org/wiki/Connected-component_labeling) in order to assign a different (integer) index to each component:

In [ ]:
labeled, nr_objects = mh.label(bin_image)
print(nr_objects)

plt.imshow(labeled)
plt.jet()

## Separating touching cells

We now got the cells nicely separated from the background, but we have also merged several cells together.

The general strategy we can use to separate the cells is the following:

1. Smooth the image with a Gaussian filter (we need to specify the  $\sigma$ parameter)
2. Find regional maxima on this smoothed image to identify each cell
3. Use [watershed](http://en.wikipedia.org/wiki/Watershed_(image_processing%29) on the [distance transformed image](http://en.wikipedia.org/wiki/Distance_transform) to separate cells.

We are going to use an interactive mode function to find out the best value for $\sigma$:

In [ ]:
@interact(sigma=(1.,16.))
def check_sigma(sigma):
    dnaf = mh.gaussian_filter(dna.astype(float), sigma)
    maxima = mh.regmax(mh.stretch(dnaf))
    maxima = mh.dilate(maxima, np.ones((5,5)))
    plt.imshow(mh.as_rgb(np.maximum(255*maxima, dnaf), dnaf, dna > T_mean))

It seems $\sigma = 12$ is a pretty good guess:

In [ ]:
sigma = 12.0
dnaf = mh.gaussian_filter(dna.astype(float), sigma)
maxima = mh.regmax(mh.stretch(dnaf))
maxima,_= mh.label(maxima)
plt.imshow(maxima)

Now, we compute the distance transform. Again, this is a single call, `mh.distance`:

In [ ]:
dist = mh.distance(bin_image)
plt.imshow(dist)

Because of the way that the watershed function is defined in mahotas, we need to invert the distance transform. Also, for technical reasons, we convert to `uint8`.

Finally, we can call `mh.cwatershed` with the `dist` image and the `maxima` as seeds.

In [ ]:
dist = 255 - mh.stretch(dist)
watershed = mh.cwatershed(dist, maxima)
plt.imshow(watershed)

We now take away the background to make things nicer:

In [ ]:
watershed *= bin_image
plt.imshow(watershed)

# Cleaning up regions

Almost there.

Let's first remove cells touching the border (the module `mh.labeled` has [several functions](http://mahotas.readthedocs.org/en/latest/labeled.html?highlight=labeled) to deal with labeled images):

In [ ]:
watershed = mh.labeled.remove_bordering(watershed)
plt.imshow(watershed)

Let us also remove things that are too small to be a cell. Again, we use an interactive display to choose the parameters:

In [ ]:
sizes = mh.labeled.labeled_size(watershed)

# The conversion below is not necessary in newer versions of mahotas:
watershed = watershed.astype(np.intc)

@interact(min_size=(100,4000,20))
def do_plot(min_size):
    filtered = mh.labeled.remove_regions_where(watershed, sizes < min_size)
    print("filtering {}...".format(min_size))
    plt.imshow(filtered)

2000 seems like it works.

So, we remove these regions and relabel.

Relabeling is necessary because the `remove_regions_where` call only sets the removed regions to zero, relabeling is necessary to make the result be in the range `0 .. N`. Relabel also returns the number of objects that are left.

In [ ]:
min_size = 2000
filtered = mh.labeled.remove_regions_where(watershed, sizes < min_size)

labeled,nr_objects = mh.labeled.relabel(filtered)
print("Number of cells: {}".format(nr_objects))